In [1]:
import sys
import numpy as np
import pandas as pd
import pickle
import json

In [2]:
from sklearn.model_selection import train_test_split

from sktime.classification.dictionary_based import IndividualBOSS, ContractableBOSS, BOSSEnsemble
#from sktime.classification.hybrid import HIVECOTEV1
from sktime.classification.kernel_based import ROCKETClassifier
from sktime.classification.shapelet_based import ShapeletTransformClassifier
from sktime.contrib.vector_classifiers._rotation_forest import RotationForest
from sktime.classification.interval_based import RandomIntervalSpectralForest, TimeSeriesForestClassifier
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier

In [3]:
from sktime.classification.kernel_based import ROCKETClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.pipeline import make_pipeline

from sktime.transformations.panel.rocket import Rocket

In [4]:
json_data = []
SERIES_LEN =300
with open(f'{SERIES_LEN}_data.json') as f:
    data = json.load(f)

In [5]:
from copy import deepcopy

In [6]:
data.keys()

dict_keys(['MW', 'TV', 'Oil Heater', 'HD', 'WM', 'IR', 'FR', 'MG', 'EK', 'GY'])

In [7]:
X=[]
y=[]
for curr_key,curr_val in data.items():
    print(curr_key)
    for curr_arr in curr_val:
        y.append(curr_key)
        X.append(deepcopy(curr_arr))

MW
TV
Oil Heater
HD
WM
IR
FR
MG
EK
GY


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y,shuffle=True,random_state=5,stratify=y, train_size=0.67)

In [9]:
def get_correct_format(arr):
    ret_arr=[]
    for curr_arr in arr:
        ret_arr.append([pd.Series(curr_arr)])
    return pd.DataFrame(ret_arr)

In [10]:
X_train=get_correct_format(X_train)

In [11]:
X_test=get_correct_format(X_test)

In [12]:
len(X_train)

1393

In [13]:
y_train=np.array(y_train)
y_test=np.array(y_test)

### Initialise ROCKET and Transform the Training Data¶


In [14]:
rocket = Rocket()  # by default, ROCKET uses 10,000 kernels
rocket.fit(X_train)
X_train_transform = rocket.transform(X_train)

###  Fit a Classifier

In [15]:
np.logspace(-3, 3, 10)

array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03])

In [16]:
classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10), normalize=True)
classifier.fit(X_train_transform, y_train)

/home/anmolagarwal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), _RidgeGCV())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  FutureWarning,


RidgeClassifierCV(alphas=array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03]),
                  normalize=True)

## Dealing with test data

In [17]:
X_test_transform = rocket.transform(X_test)

In [18]:
classifier.score(X_test_transform, y_test)

0.9082969432314411

In [21]:
given=classifier.predict(X_test_transform)

In [22]:
exp=y_test

In [24]:
len(exp)

687

In [25]:
len(given)

687

In [30]:
def get_freq(arr):
    curr_dict={}
    for x in arr:
        try:
            curr_dict[x]+=1
        except:
            curr_dict[x]=1
    return curr_dict

In [31]:
get_freq(exp)

{'WM': 178,
 'Oil Heater': 152,
 'EK': 33,
 'IR': 33,
 'TV': 139,
 'FR': 73,
 'GY': 20,
 'MW': 13,
 'HD': 40,
 'MG': 6}

In [34]:
correct_predict={}
tot=0

In [35]:
for i in range(687):
    exp_label=exp[i]
    got_label=given[i]
    if exp_label==got_label:
        tot+=1
        try:
            correct_predict[exp_label]+=1
        except:
            correct_predict[exp_label]=1

In [36]:
correct_predict

{'WM': 174,
 'Oil Heater': 150,
 'EK': 14,
 'IR': 10,
 'TV': 139,
 'FR': 72,
 'GY': 19,
 'MW': 10,
 'HD': 36}